# Primal 1.0
Redone from the first section of Prof. Kruczenski's Maple script file `K_sp_leaf_points.maple`.

Remarks:
- At first glance, it looks like the data points generately using `scipy` alone are not **as accurate as** Maple, but seem to be **accurate enough** (hopefully).
- Implementing `sympy` will grant us the ability to manipulate the variables/data to an arbitrary precision, like Maple and Mathematica.
- In the case where the data generated using Python is pretty far off the more precise values generated in Maple/Mathematica, we will use Python solely for importing the data files and optimizing with `cvxpy`, without generating data in Python.

In [1]:
import os
import numpy as np
from scipy.special import lqn

# import scipy as sp
# import sympy as smp
# import cvxpy as cp
# import matplotlib.pyplot as plt
# import mpmath as mp

## Numerical Implementation
For the primal problem, we have to compute the primal functional and the partial waves to impose unitarity. For that purpose, we map the region $s \in [4, \infty)$ to $\xi \in [0, \pi]$ using
$$
s = \frac{4}{\cos^2 \left( \frac{\xi}{2} \right)}.
$$

Through a redefinition of the constant $f_0$ and the spectral density $\sigma(x)$, we can rewrite the amplitude $F(s, t, u)$ in a neater way, visualized later, where $F$ depends on the kernel $\mathcal{K}$, given by
$$
\mathcal{K}(\xi, s) = \frac{1}{\pi} \frac{\sin(\xi)}{\frac{8}{s} - 1 - \cos(\xi)}.
$$

To compute the partial waves, it is useful to define
$$
\begin{align*}
    \hat{\mathbf{\Phi}}_{\ell}(s; \xi_{1}) & = \int_{-1}^{+1} P_{\ell}(\mu) \mathcal{K}(\xi, t) ~ \mathrm{d} \mu, \\
    \tilde{\mathbf{\Phi}}_{\ell}(s; \xi_{1}, \xi_{2}) & = \int_{-1}^{+1} P_{\ell}(\mu) \mathcal{K}(\xi, t) \mathcal{K}(\xi, u) ~ \mathrm{d} \mu, \\
\end{align*}
$$
where, as usual, $t$ and $u$ are function of $\mu$.

### Interpolation Points

We will discretize $\xi$ by choosing equally space points, such that
$$
\xi_j = j \Delta_{\xi}, \quad \Delta_{\xi} = \frac{\pi}{M}, \quad 0 \leq j \leq M, \quad s_j = \frac{4}{\cos^2(\frac{\xi_j}{2})}.
$$

Specifically, the grid will be divided into $2M$ pieces, where $\xi_j$ will take on values in the range $[0, 2 \pi)$ in segments of $\frac{\pi}{M}$.

The primal functional becomes
$$
\mathcal{F}_P = a f_0 + a_{j_1} \sigma_{j_1} + a_{j_1, j_2} \rho_{j_1, j_2},
$$
where the coefficients are
$$
\begin{align*}
    a & = 1, \\
    a_{j_1} & = \Delta_{\xi} \left( \mathcal{K}(\xi_{j_1}, s_0) + \mathcal{K}(\xi_{j_1}, t_0) + \mathcal{K}(\xi_{j_1}, u_0) \right), \\
    a_{j_1, j_2} & = \frac{\Delta_{\xi}^2}{2} \left( \mathcal{K}(\xi_{j_1}, s_0) \mathcal{K}(\xi_{j_2}, t_0) + \mathcal{K}(\xi_{j_1}, s_0) \mathcal{K}(\xi_{j_2}, u_0) + \mathcal{K}(\xi_{j_1}, t_0) \mathcal{K}(\xi_{j_2}, u_0) \right) + (j_1 \leftrightarrow j_2),
\end{align*}
$$
and the partial waves become
$$
h_{\ell j} = h_{\ell j} f_0 + h_{\ell j; j_1} \sigma_{j_1} + h_{\ell j; j_1 j_2} \rho_{j_1 j_2}
$$
where the coefficients are
$$
\begin{align*}
    h_{\ell j} & = \frac{\pi}{2} \sin \left( \frac{\xi_j}{2} \right) \delta_{\ell 0}, \\
    h_{\ell j; j_1} & = \frac{\pi}{2} \sin \left( \frac{\xi_j}{2} \right) \left( \delta_{\ell 0} \mathcal{K}(\xi_{j_1}, s_j^+) + \Delta_{\xi} \hat{\mathbf{\Phi}}{_\ell}(s_j; \xi_{j_1}) \right), \\
    h_{\ell j; j_1 j_2} & = \frac{\pi}{2} \sin \left( \frac{\xi_j}{2} \right) \left( \Delta_{\xi} \mathcal{K}(\xi_{j_1}, s_j^+) \hat{\mathbf{\Phi}}_{\ell}(s_j; \xi_{j_2}) + \frac{\Delta_{\xi}^2}{2} \tilde{\mathbf{\Phi}}_{\ell}(s_j; \xi_{j_1}, \xi_{j_2}) \right) + (j_1 \leftrightarrow j_2),
\end{align*}
$$
where
$$
\mathcal{K}(\xi_{j_1}, s_j^+) = \hat{K}_{j j_1} + \frac{i}{2\pi} \delta_{j j_1}.
$$

The matrix $\hat{K}_{j_1 j_2}$ implements the principal part integral
$$
f(\xi) = \frac{1}{\pi} \int_0^\pi \frac{\sin(\xi_1)}{\cos(\xi) - \cos(\xi_1)} g(\xi_1) ~ \mathrm{d} \xi_1.
$$

To implement this integral, we extend $g(\xi_1)$ to the range $- \pi \leq \xi_1 \leq \pi$ by assuming that it is antisymmetric, *i.e.* $g(-\xi_1) = - g(\xi_1)$, so that the integrand is symmetric.

For the spectral densities $\sigma(\xi_1)$ and $\rho(\xi_1, \xi_2)$, this agrees with them being the imaginary part of the amplitude that changes sign across the cut.

Now, discretizing $f(\xi)$, we have
$$
f(\xi_j) = \sum_{j_1 = - M}^{M} \hat{K}_{j j_1} g(\xi_{j1}),
$$
where
$$
\hat{K}_{j_1 j_2} = \frac{1}{2M} \left( 1 - (-1)^{j_1 - j_2} \right) \cot \left( \frac{\pi}{2M} (j_1 - j_2) \right).
$$

In [43]:
M = 5  # Number of interpolation points
xi = np.linspace(0, 2 * np.pi, 2 * M, endpoint=False)  # Interpolation points
s = 4 / np.cos(xi / 2) ** 2  # Mapping from xi to s

We will now compute the real part of the kernel $\mathcal{K}$, which is $\hat{K}_{j_1 j_2}$, given by
$$
\hat{K}_{j_1 j_2} = \frac{1}{2M} \left( 1 - (-1)^{j_1 - j_2} \right) \cot \left( \frac{\pi}{2M} (j_1 - j_2) \right).
$$

In [44]:
K = np.zeros((2 * M, 2 * M))
for j1 in range(0, 2 * M):
    for j2 in range(j1, 2 * M):
        K[j1, j2] = (
            1
            / (2 * M)
            * (1 - (-1) ** (j1 - j2))
            * 1
            / np.tan(np.pi / (2 * M) * (j1 - j2))
        )
        K[j2, j1] = -K[j1, j2]

        np.nan_to_num(K, copy=False)  # Replace NaN with 0

C:\Users\Razz\AppData\Local\Temp\ipykernel_14300\4069183133.py:4: RuntimeWarning: invalid value encountered in double_scalars
  K[j1, j2] = 1/(2*M) * (1 - (-1)**(j1 - j2)) * 1/np.tan(np.pi/(2*M) * (j1 - j2))


Now, we will compute the kernel in a different method by dividing the kernel $\mathcal{K}$ into its real and imaginary parts.

In [45]:
K_i = np.zeros((M + 1, M + 1))  # imaginary
K_d = np.zeros((M + 1, M + 1))  # real? delta?
for j1 in range(0, M + 1):
    for j2 in range(1, M):
        K_i[j1, j2] = -(K[j1, j2] - K[j1, 2 * M - j2])
        K_d[j1, j2] = K[j1, j2] + K[j1, 2 * M - j2]

for j1 in range(0, M + 1):
    K_d[j1, 0] = K[j1, 0]
    K_d[j1, M] = K[j1, M]


fvra = np.cos(4 * xi)
fvia = np.sin(4 * xi)
fviaN = np.dot(K, fvra)
norm1 = np.linalg.norm(fvia - fviaN)
print("norm1:", norm1)

fvr = np.cos(4 * xi)[: M + 1]
fvi = np.sin(4 * xi)[: M + 1]
fvrN = np.dot(K_i, fvi)
fviN = np.dot(K_d, fvr)
norm2 = np.linalg.norm(fvr - fvrN)
norm3 = np.linalg.norm(fvi - fviN)
print("norm2:", norm2)
print("norm3:", norm3)

norm1: 8.475324094644878e-16
norm2: 3.885780586188048e-16
norm3: 8.118219146209984e-16


Now, we will take our initial values, defined by $\ell_{\text{max}}$, $s_0$, $t_0$, and $u_0$, such that
$$
    \ell_{max} = 20, \quad s_0 = t_0 = u_0 = \frac{4}{3}.
$$

In [46]:
lmax = 20
s0, t0, u0 = 4 / 3, 4 / 3, 4 / 3


def mathcalK(xi, s):
    return 1 / np.pi * np.sin(xi) / (8 / s - 1 - np.cos(xi))


as1 = mathcalK(xi, s0)
at1 = mathcalK(xi, t0)
au1 = mathcalK(xi, u0)

a0 = 1
a1 = as1 + at1 + au1
a2 = np.outer(as1, at1) + np.outer(as1, au1) + np.outer(at1, au1)
a2 = 1 / 2 * (a2 + a2.T)

We will now define the Legendre function of the second kind $Q_n$ with the following argument.
$$
    Q(\ell, \xi_1, \xi) \equiv Q_n \left( \ell, 1 + \frac{4 \left( 1 + \cos(\xi) \right)}{ \left( 1 - \cos(\xi) \right) \left( 1 + \cos(\xi_1) \right)} \right).
$$

In [47]:
def legendreQ(l, xi_1, xi):
    return lqn(l, 1 + 4 * (1 + np.cos(xi)) / ((1 - np.cos(xi)) * (1 + np.cos(xi_1))))


legendreQMatrix = np.zeros((lmax + 1, M + 1, M + 1))
print(np.shape(legendreQMatrix))
for l in range(0, lmax + 1):
    for j1 in range(1, M + 1):
        for j2 in range(2, M + 1):
            legendreQMatrix[l, j1 - 1, j2 - 1] = legendreQ(
                2 * (l + 1), xi[j1 - 1], xi[j2 - 1]
            )[0][2 * l]
            np.nan_to_num(legendreQMatrix, copy=False)  # Replace NaN with 0

            # print("l:", l)
            # print("j1:", j1)
            # print("j2:", j2)
            # print("leg:", legendreQ(2*(l + 1), xi[j1 - 1], xi[j2 - 1])[0][2*l])

print(legendreQMatrix)

(21, 6, 6)
[[[0.00000000e+000 5.01817899e-002 2.11935356e-001 5.31393615e-001
   1.17435901e+000 0.00000000e+000]
  [0.00000000e+000 4.56015618e-002 1.95161357e-001 4.99124354e-001
   1.12919263e+000 0.00000000e+000]
  [0.00000000e+000 3.34077917e-002 1.48379688e-001 4.03219784e-001
   9.87012281e-001 0.00000000e+000]
  [0.00000000e+000 1.79125294e-002 8.37615244e-002 2.51751992e-001
   7.26104540e-001 0.00000000e+000]
  [0.00000000e+000 5.01541439e-003 2.45886308e-002 8.31391494e-002
   3.22111984e-001 0.00000000e+000]
  [0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
   0.00000000e+000 0.00000000e+000]]

 [[0.00000000e+000 1.68430590e-005 1.26115939e-003 1.92295146e-002
   1.80056594e-001 0.00000000e+000]
  [0.00000000e+000 1.26400528e-005 9.85740787e-004 1.60082122e-002
   1.62122117e-001 0.00000000e+000]
  [0.00000000e+000 4.97064567e-006 4.34208183e-004 8.54235013e-003
   1.12425788e-001 0.00000000e+000]
  [0.00000000e+000 7.66283679e-007 7.82775739e-005 2.1083401

With that definition, we can now rewrite $\hat{\mathbf{\Phi}}$ as
$$
    \hat{\mathbf{\Phi}} (\ell, \xi_1, \xi) = \sin(\xi_1) \left( - \frac{2}{1 + \cos(\xi_1)} \delta_{\ell 0} + \frac{8 \left( 1 + \cos(\xi) \right)}{ \left( 1 + \cos(\xi_1) \right)^2 \left( 1 - \cos(\xi) \right)} Q(\ell, \xi, \xi_1) \right).
$$

In [48]:
def PhiHat(l, xi_1, xi, Q):
    if l == 0:
        return np.sin(xi_1) * (
            -2 / (1 + np.cos(xi_1))
            + 8 * (1 + np.cos(xi)) / ((1 + np.cos(xi_1)) ** 2 * (1 - np.cos(xi))) * Q
        )
    else:
        return np.sin(xi_1) * (
            8 * (1 + np.cos(xi)) / ((1 + np.cos(xi_1)) ** 2 * (1 - np.cos(xi))) * Q
        )

    # return np.sin(xi_1) * ( (l == 0) * (-2/(1 + np.cos(xi_1))) + 8*(1 + np.cos(xi)) / ((1 + np.cos(xi_1))**2 * (1 - np.cos(xi))))


PhiHatMatrix = np.zeros((lmax + 1, M + 1, M + 1))
print(np.shape(PhiHatMatrix))
for l in range(0, lmax + 1):
    for j1 in range(1, M + 1):
        for j2 in range(2, M + 1):
            PhiHatMatrix[l, j1 - 1, j2 - 1] = PhiHat(
                l, xi[j1 - 1], xi[j2 - 1], legendreQMatrix[l, j1 - 1, j2 - 1]
            )
            np.nan_to_num(PhiHatMatrix, copy=False)  # Replace NaN with 0

            # print("l:", l)
            # print("j1:", j1)
            # print("j2:", j2)
            # print("PhiHat:", PhiHat(l, xi[j1 - 1], xi[j2 - 1]) * legendreQMatrix[l, j1 - 1, j2 - 1])

print(PhiHatMatrix)

(21, 6, 6)
[[[ 0.00000000e+000 -0.00000000e+000 -0.00000000e+000 -0.00000000e+000
   -0.00000000e+000  0.00000000e+000]
  [ 0.00000000e+000 -2.91833061e-002 -1.18594060e-001 -2.71262396e-001
   -4.78544864e-001  0.00000000e+000]
  [ 0.00000000e+000 -4.80038165e-002 -2.04959962e-001 -5.08002697e-001
   -9.90405429e-001  0.00000000e+000]
  [ 0.00000000e+000 -4.90145533e-002 -2.24140914e-001 -6.35102165e-001
   -1.53120945e+000  0.00000000e+000]
  [ 0.00000000e+000 -3.08201052e-002 -1.50111585e-001 -4.97576305e-001
   -1.77128996e+000  0.00000000e+000]
  [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  0.00000000e+000
    0.00000000e+000  0.00000000e+000]]

 [[ 0.00000000e+000  0.00000000e+000  0.00000000e+000  0.00000000e+000
    0.00000000e+000  0.00000000e+000]
  [ 0.00000000e+000  1.72036338e-004  2.68326783e-003  1.21419458e-002
    2.45933518e-002  0.00000000e+000]
  [ 0.00000000e+000  2.09057846e-004  3.65242802e-003  2.00218956e-002
    5.27015952e-002  0.00000000e+000]
  [ 0.

Additionally, we can rewrite $\tilde{\mathbf{\Phi}}$ as
$$
    \tilde{\mathbf{\Phi}} (\ell, \xi, \xi_1) = \sin(\xi_1) \left( - \frac{2}{1 + \cos(\xi_1)} \delta_{\ell 0} + \frac{8 (1 + \cos(\xi))}{(1 + \cos(\xi_1))^2 (1 - \cos(\xi))} \right)
$$

In [49]:
def PhiTilde(l, xi_1, xi):
    return np.sin(xi_1) * (
        (l == 0) * (-2 / (1 + np.cos(xi_1)))
        + 8 * (1 + np.cos(xi)) / ((1 + np.cos(xi_1)) ** 2 * (1 - np.cos(xi)))
    )


# av1 = 1 + 4 * (1 + np.cos(xi)) / ( (1 + np.cos(xi_1)) * (1 - np.cos(xi)) )
# bv1 = 1 + 4 * (1 + np.cos(xi)) / ( (1 + np.cos(xi_2)) * (1 - np.cos(xi)) )

PhiTildeMatrix = np.zeros((lmax + 1, M + 1, M + 1))
print(np.shape(PhiTildeMatrix))
for l in range(0, lmax + 1):
    for j1 in range(1, M + 1):
        for j2 in range(2, M + 1):
            PhiTildeMatrix[l, j1 - 1, j2 - 1] = PhiTilde(l, xi[j1 - 1], xi[j2 - 1])
            np.nan_to_num(PhiTildeMatrix, copy=False)  # Replace NaN with 0

            # print("l:", l)
            # print("j1:", j1)
            # print("j2:", j2)
            # print("PhiTilde:", PhiTilde(l, xi[j1 - 1], xi[j2 - 1]))

print(PhiTildeMatrix)

(21, 6, 6)
[[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   -0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  1.29605735e+01  2.07224319e+00  1.08642924e-01
   -4.98142929e-01  0.00000000e+00]
  [ 0.00000000e+00  4.06054039e+01  6.95861274e+00  8.90754203e-01
   -9.84317204e-01  0.00000000e+00]
  [ 0.00000000e+00  1.48189029e+02  2.74355946e+01  5.65893395e+00
   -1.07042428e+00  0.00000000e+00]
  [ 0.00000000e+00  1.21498939e+03  2.38073583e+02  6.18966976e+01
    7.45504586e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00]]

 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  1.36104129e+01  2.72208259e+00  7.58482317e-01
    1.51696463e-01  0.00000000e+00]
  [ 0.00000000e+00  4.20584890e+01  8.41169779e+00  2.34383926e+00
    4.68767852e-01  0.00000000e+00]
  [ 0.00000000e+00  1.50941792e+02  3.01883585e+01  8.411697

Save the data points in the file.

In [51]:
Delta = np.pi / M


def delta(x, y):
    return 1 if x == y else 0


Mfl = (lmax + 1) * (M + 1)
A1h = np.zeros(M + 1)
A2h = np.zeros((M + 1) * (M + 1))
hl1_0M = np.zeros((M + 1, M + 1))
hl1_lM = np.zeros((Mfl, M + 1))
hl2_IM = np.zeros((Mfl, (M + 1) * (M + 1)))
hl2_RM = np.zeros((Mfl, (M + 1) * (M + 1)))

# Now we put computation data in matrix form
for j in range(0, M):
    A1h[j] = a1[j] * Delta

for j1 in range(0, M):
    for j in range(0, M):
        hl1_0M[j, j1] = np.pi / 2 * np.sin(xi[j] / 2) * K_i[j, j1]

for l in range(0, lmax + 1):
    for j in range(0, M):
        for j1 in range(0, M):
            jf1 = l * (M + 1) + j
            jf2 = j1
            hl1_lM[jf1, jf2] = (
                1 / 2 * np.sin(xi[j] / 2) * Delta * PhiHatMatrix[l, j1, j]
            )

for j1 in range(0, M):
    for j2 in range(0, M):
        jf2 = j1 * (M + 1) + j2
        A2h[jf2] = a2[j1, j2] * Delta**2

for l in range(0, lmax + 1):
    for j in range(0, M):
        for j1 in range(0, M):
            for j2 in range(0, M):
                jf1 = l * (M + 1) + j
                jf2 = j1 * (M + 1) + j2
                hl2_RM[jf1, jf2] = (
                    np.pi
                    / 2
                    * np.sin(xi[j] / 2)
                    * (
                        K_i[j, j1] * Delta * PhiHatMatrix[l, j2, j]
                        + K_i[j, j2] * Delta * PhiHatMatrix[l, j1, j]
                        + Delta**2 * PhiTildeMatrix[l, j1, j2]
                    )
                )

for l in range(0, lmax + 1):
    for j in range(0, M):
        for j1 in range(0, M):
            for j2 in range(0, M):
                jf1 = l * (M + 1) + j
                jf2 = j1 * (M + 1) + j2
                hl2_IM[jf1, jf2] = (
                    1
                    / 4
                    * np.sin(xi[j] / 2)
                    * (
                        delta(j, j1) * Delta * PhiHatMatrix[l, j2, j]
                        + delta(j, j2) * Delta * PhiHatMatrix[l, j1, j]
                    )
                )

print("Done")

Done
Coefficients saved.


Save the data in a text file.

In [ ]:
currentdir = os.getcwd()
dataKFile = f"{currentdir}/primal_1.0_points_data_P2_nopole_lmax{lmax}_M{M}.dat"

with open(dataKFile, "w") as fd:
    for j in range(0, M + 1):
        fd.write(f"{xi[j]:4.20g}\n")

    for j in range(0, M + 1):
        fd.write(f"{A1h[j]:4.20g}\n")

    for jf2 in range(0, M + 1):
        for jf1 in range(0, M + 1):
            fd.write(f"{hl1_0M[jf1, jf2]:4.20g}\n")

    for jf2 in range(0, M + 1):
        for jf1 in range(0, Mfl):
            fd.write(f"{hl1_lM[jf1, jf2]:4.20g}\n")

    for j in range(0, (M + 1) * (M + 1)):
        fd.write(f"{A2h[j]:4.20g}\n")

    for jf2 in range(0, (M + 1) * (M + 1)):
        for jf1 in range(0, Mfl):
            fd.write(f"{hl2_RM[jf1, jf2]:4.20g}\n")

    for jf2 in range(0, (M + 1) * (M + 1)):
        for jf1 in range(0, Mfl):
            fd.write(f"{hl2_IM[jf1, jf2]:4.20g}\n")

print("Coefficients saved.")

## VARIATION WITH RANGE

In [52]:
Delta = np.pi / M


def delta(x, y):
    return 1 if x == y else 0


Mfl = (lmax + 1) * (M + 1)
print(Mfl)
A1h = np.zeros(M + 1)
A2h = np.zeros((M + 1) * (M + 1))
hl1_0M = np.zeros((M + 1, M + 1))
hl1_lM = np.zeros((Mfl, M + 1))
hl2_IM = np.zeros((Mfl, (M + 1) * (M + 1)))
hl2_RM = np.zeros((Mfl, (M + 1) * (M + 1)))

# Now we put computation data in matrix form
for j in range(1, M + 1):
    A1h[j - 1] = a1[j - 1] * Delta

for j1 in range(1, M + 1):
    for j in range(1, M + 1):
        hl1_0M[j - 1, j1 - 1] = np.pi / 2 * np.sin(xi[j - 1] / 2) * K_i[j - 1, j1 - 1]

for l in range(0, lmax + 1):
    for j in range(1, M + 1):
        for j1 in range(1, M + 1):
            jf1 = l * (M + 1) + j
            jf2 = j1
            hl1_lM[jf1 - 1, jf2 - 1] = (
                1
                / 2
                * np.sin(xi[j - 1] / 2)
                * Delta
                * PhiHatMatrix[l - 1, j1 - 1, j - 1]
            )

for j1 in range(1, M + 1):
    for j2 in range(1, M + 1):
        jf2 = j1 * (M + 1) + j2
        A2h[jf2 - 1] = a2[j1 - 1, j2 - 1] * Delta**2

for l in range(0, lmax + 1):
    for j in range(1, M + 1):
        for j1 in range(1, M + 1):
            for j2 in range(1, M + 1):
                jf1 = l * (M + 1) + j
                jf2 = j1 * (M + 1) + j2
                hl2_RM[jf1 - 1, jf2 - 1] = (
                    np.pi
                    / 2
                    * np.sin(xi[j - 1] / 2)
                    * (
                        K_i[j - 1, j1 - 1] * Delta * PhiHatMatrix[l - 1, j2 - 1, j - 1]
                        + K_i[j - 1, j2 - 1]
                        * Delta
                        * PhiHatMatrix[l - 1, j1 - 1, j - 1]
                        + Delta**2 * PhiTildeMatrix[l - 1, j1 - 1, j2 - 1]
                    )
                )

for l in range(0, lmax + 1):
    for j in range(1, M + 1):
        for j1 in range(1, M + 1):
            for j2 in range(1, M + 1):
                jf1 = l * (M + 1) + j
                jf2 = j1 * (M + 1) + j2
                hl2_IM[jf1 - 1, jf2 - 1] = (
                    1
                    / 4
                    * np.sin(xi[j - 1] / 2)
                    * (
                        delta(j - 1, j1 - 1)
                        * Delta
                        * PhiHatMatrix[l - 1, j2 - 1, j - 1]
                        + delta(j - 1, j2 - 1)
                        * Delta
                        * PhiHatMatrix[l - 1, j1 - 1, j - 1]
                    )
                )

print("Done")

126
Done
Coefficients saved.


In [ ]:
currentdir = os.getcwd()
dataKFile = f"{currentdir}/SmQCD_primalE_points_data_P2_nopole_lmax{lmax}_M{M}.dat"

with open(dataKFile, "w") as fd:
    for j in range(0, M + 1):
        fd.write(f"{xi[j]:4.20g}\n")

    for j in range(0, M + 1):
        fd.write(f"{A1h[j]:4.20g}\n")

    for jf2 in range(0, M + 1):
        for jf1 in range(0, M + 1):
            fd.write(f"{hl1_0M[jf1, jf2]:4.20g}\n")

    for jf2 in range(0, M + 1):
        for jf1 in range(0, Mfl):
            fd.write(f"{hl1_lM[jf1, jf2]:4.20g}\n")

    for j in range(0, (M + 1) * (M + 1)):
        fd.write(f"{A2h[j]:4.20g}\n")

    for jf2 in range(0, (M + 1) * (M + 1)):
        for jf1 in range(0, Mfl):
            fd.write(f"{hl2_RM[jf1, jf2]:4.20g}\n")

    for jf2 in range(0, (M + 1) * (M + 1)):
        for jf1 in range(0, Mfl):
            fd.write(f"{hl2_IM[jf1, jf2]:4.20g}\n")

print("Coefficients saved.")

## Notes
A relation between the two solutions $P_n(x)$ and $Q_n(x)$ of the Legendre differential equation is given by
$$
Q_n(x) = P_n(x) \int \frac{1}{P_n^2(x) (1 - x^2)} ~ \mathrm{d} x.
$$